In [1]:
import subprocess
import glob
import os
import tempfile
import base64

### **Funciones de apoyo**

In [2]:
def ejecutar_adb(comando):
    try:
        resultado = subprocess.run(
            comando, 
            shell=True, 
            capture_output=True, 
            text=True, 
            timeout=60
        )
        return resultado.returncode == 0, resultado.stdout + resultado.stderr
    except Exception as e:
        return False, str(e)

def instalar_apk_con_paquete(ruta_apk):
    if not os.path.exists(ruta_apk):
        return False, None
    
    exito, apps_antes = ejecutar_adb('adb shell "pm list packages -3"')
    apps_antes = set([p.replace('package:', '').strip() for p in apps_antes.split('\n') if p])
    
    exito, mensaje = ejecutar_adb(f'adb install "{ruta_apk}"')
    
    if not exito:
        return False, None
    
    exito, apps_despues = ejecutar_adb('adb shell "pm list packages -3"')
    apps_despues = set([p.replace('package:', '').strip() for p in apps_despues.split('\n') if p])
    
    nuevas_apps = apps_despues - apps_antes
    
    if nuevas_apps:
        paquete = list(nuevas_apps)[0] 
        return True, paquete
    else:
        return True, None

def desinstalar_app(paquete='me.talkyou.app.im', mantener_datos=False):
    comando = f"adb uninstall {'-k ' if mantener_datos else ''}{paquete}"
    exito, mensaje = ejecutar_adb(comando)
    
    if exito or "Success" in mensaje:
        return True
    else:
        return False

def listar_aplicaciones(filtro=None):
    if filtro:
        comando = f'adb shell "pm list packages | grep {filtro}"'
    else:
        comando = "adb shell pm list packages"
    
    exito, mensaje = ejecutar_adb(comando)
    if exito:
        paquetes = [linea.replace('package:', '').strip() 
                   for linea in mensaje.split('\n') 
                   if linea.startswith('package:')]
        return True, paquetes
    else:
        return False, mensaje

### **Listado de aplicaciones a utilizar**

In [3]:
lista_apks = sorted(glob.glob('./apks/*'))
lista_apks

['./apks/me.talkyou.app.im_3.4.1.apk',
 './apks/me.talkyou.app.im_4.16.5.apk',
 './apks/me.talkyou.app.im_5.0.0.apk',
 './apks/me.talkyou.app.im_6.1.5.apk',
 './apks/me.talkyou.app.im_6.1.6.apk',
 './apks/me.talkyou.app.im_6.1.7.apk',
 './apks/me.talkyou.app.im_6.1.8.apk',
 './apks/me.talkyou.app.im_6.1.9.apk',
 './apks/me.talkyou.app.im_6.2.0.apk',
 './apks/me.talkyou.app.im_6.2.1.apk',
 './apks/me.talkyou.app.im_6.2.2.apk']

### **Verificación de nombre de paquetes de las aplicación**

In [63]:
lista_apk_paquete = []
for apk in lista_apks:
    sucess, paquete = instalar_apk_con_paquete(apk)
    if sucess and paquete is not None:
        desinstalar_app()
    lista_apk_paquete.append((apk, paquete))
    print(apk, paquete, sep='------------------> ')

./apks/me.talkyou.app.im_3.4.1.apk------------------> me.talkyou.app.im
./apks/me.talkyou.app.im_4.16.5.apk------------------> me.talkyou.app.im
./apks/me.talkyou.app.im_5.0.0.apk------------------> me.talkyou.app.im
./apks/me.talkyou.app.im_6.1.5.apk------------------> me.talkyou.app.im
./apks/me.talkyou.app.im_6.1.6.apk------------------> me.talkyou.app.im
./apks/me.talkyou.app.im_6.1.7.apk------------------> me.talkyou.app.im
./apks/me.talkyou.app.im_6.1.8.apk------------------> me.talkyou.app.im
./apks/me.talkyou.app.im_6.1.9.apk------------------> me.talkyou.app.im
./apks/me.talkyou.app.im_6.2.0.apk------------------> me.talkyou.app.im
./apks/me.talkyou.app.im_6.2.1.apk------------------> me.talkyou.app.im
./apks/me.talkyou.app.im_6.2.2.apk------------------> me.talkyou.app.im


## **Procedimiento manual** 
**Algoritmo**

Desde aplicacion numero 1 hasta n aplicaciones hacer 

1. Instalar aplicación 

2. Registro en aplicación

3. Remplazar numero registrado por numero sugerido en archivos de configuración de la aplicación 

4. Verificación de configuración nueva en aplicacion y pruebas 

5. Desintalar aplicación


### **1. Instalacion de apk individual**

In [19]:
# Numero de aplicaciones = 11
# Pendiente 2
idx_apk = 0
sucess, paquete = instalar_apk_con_paquete(lista_apks[idx_apk])
print(lista_apks[idx_apk], paquete, sep=' ------------------> ')

./apks/me.talkyou.app.im_3.4.1.apk ------------------> me.talkyou.app.im


### **2. Remplazo de numero viejo por nuevo numero**

In [ ]:
def reemplazar_simple_corregido():
    viejo = "5522958640"
    nuevo = "5511282552"
    ruta = "/data/data/me.talkyou.app.im"

    print("🔍 Buscando archivos...")
    cmd_buscar = f'adb shell "su -c grep -rl {viejo} {ruta}"'
    result = subprocess.run(cmd_buscar, shell=True, capture_output=True, text=True)
    
    archivos = []
    for linea in result.stdout.split('\n'):
        linea = linea.strip()
        if linea and not linea.startswith('Binary file') and 'matches' not in linea:
            archivo = linea.split(':')[0] if ':' in linea else linea
            if archivo and archivo not in archivos:
                archivos.append(archivo)
    
    print(f"📁 Encontrados {len(archivos)} archivos:")
    for archivo in archivos:
        print(f"   • {archivo}")
    
    # Asegurar root
    subprocess.run("adb root", shell=True, capture_output=True)
    
    for archivo in archivos:
        print(f"\n🔄 Procesando: {archivo}")
        
        try:
            # Verificar que el archivo existe
            check_cmd = f'adb shell "su -c ls {archivo}"'
            result = subprocess.run(check_cmd, shell=True, capture_output=True, text=True)
            
            if result.returncode != 0:
                print(f"❌ Archivo no existe: {archivo}")
                continue
            
            # MÉTODO MEJORADO: Usar tempfile y pull/push para archivos binarios
            with tempfile.NamedTemporaryFile(delete=False) as temp_file:
                temp_path = temp_file.name
            
            # 1. Copiar archivo a un lugar accesible temporalmente
            temp_remote = f"/sdcard/temp_{os.path.basename(archivo)}"
            copy_cmd = f'adb shell "su -c cp \\"{archivo}\\" \\"{temp_remote}\\""'
            result_copy = subprocess.run(copy_cmd, shell=True, capture_output=True, text=True)
            
            if result_copy.returncode != 0:
                print(f"❌ Error copiando archivo: {result_copy.stderr}")
                continue
            
            # 2. Descargar usando adb pull (más confiable para binarios)
            pull_cmd = f"adb pull \"{temp_remote}\" \"{temp_path}\""
            result_pull = subprocess.run(pull_cmd, shell=True, capture_output=True, text=True)
            
            if result_pull.returncode != 0:
                print(f"❌ Error descargando: {result_pull.stderr}")
                # Limpiar archivo temporal remoto
                subprocess.run(f'adb shell "rm {temp_remote}"', shell=True)
                continue
            
            # 3. Leer y modificar archivo
            with open(temp_path, 'rb') as f:
                contenido_original = f.read()
            
            # Buscar el patrón en diferentes codificaciones
            contenido_nuevo = contenido_original
            modificado = False
            
            # Buscar en UTF-8
            if viejo.encode('utf-8') in contenido_original:
                contenido_nuevo = contenido_original.replace(viejo.encode('utf-8'), nuevo.encode('utf-8'))
                modificado = True
                print("   🔍 Patrón encontrado en UTF-8")
            
            # Buscar en formato de texto plano
            elif viejo in contenido_original.decode('latin-1', errors='ignore'):
                contenido_nuevo = contenido_original.replace(viejo.encode('latin-1'), nuevo.encode('latin-1'))
                modificado = True
                print("   🔍 Patrón encontrado en Latin-1")
            
            if not modificado:
                print(f"⚠️  No se encontró el patrón '{viejo}' en el archivo")
                # Limpiar
                os.unlink(temp_path)
                subprocess.run(f'adb shell "rm {temp_remote}"', shell=True)
                continue
            
            # 4. Guardar cambios
            with open(temp_path, 'wb') as f:
                f.write(contenido_nuevo)
            
            # 5. Subir archivo modificado
            push_cmd = f"adb push \"{temp_path}\" \"{temp_remote}\""
            result_push = subprocess.run(push_cmd, shell=True, capture_output=True, text=True)
            
            if result_push.returncode != 0:
                print(f"❌ Error subiendo: {result_push.stderr}")
                # Limpiar
                os.unlink(temp_path)
                subprocess.run(f'adb shell "rm {temp_remote}"', shell=True)
                continue
            
            # 6. Mover de vuelta al directorio original con permisos root
            move_cmd = f'adb shell "su -c cp \\"{temp_remote}\\" \\"{archivo}\\" && su -c chmod 644 \\"{archivo}\\""'
            result_move = subprocess.run(move_cmd, shell=True, capture_output=True, text=True)
            
            if result_move.returncode == 0:
                print(f"✅ Modificado exitosamente: {archivo}")
            else:
                print(f"❌ Error moviendo archivo: {result_move.stderr}")
            
            # 7. Limpiar archivos temporales
            os.unlink(temp_path)
            subprocess.run(f'adb shell "rm {temp_remote}"', shell=True)
                
        except Exception as e:
            print(f"❌ Error procesando {archivo}: {e}")
            # Limpiar en caso de error
            if 'temp_path' in locals() and os.path.exists(temp_path):
                os.unlink(temp_path)
            subprocess.run(f'adb shell "rm {temp_remote}"', shell=True, capture_output=True)

# Ejecutar
reemplazar_simple_corregido()

🔍 Buscando archivos...
📁 Encontrados 2 archivos:
   • /data/data/me.talkyou.app.im/files/mmkv/local_info
   • /data/data/me.talkyou.app.im/databases/Dingtone.db

🔄 Procesando: /data/data/me.talkyou.app.im/files/mmkv/local_info
   🔍 Patrón encontrado en UTF-8
✅ Modificado exitosamente: /data/data/me.talkyou.app.im/files/mmkv/local_info

🔄 Procesando: /data/data/me.talkyou.app.im/databases/Dingtone.db
   🔍 Patrón encontrado en UTF-8
✅ Modificado exitosamente: /data/data/me.talkyou.app.im/databases/Dingtone.db


### **Desintalación de aplicación**

In [18]:
desinstalar_app()

True